# Implementacion del metodo de la maxima verosimilitud para la regresion logistica

## Definir la funcion de entorno L(b)

## $L(\beta) = \sum_{i=1}^n P_i^{y_i}(1-Pi)^{y_i}$

In [17]:
def linkelihood(y, pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1, len(y)+1))
    for i in range(len(y)):
        sum_in[i] = np.where(y[i] ==1, pi[i], 1 -pi[i])
        total_sum = total_sum * sum_in[i]

## Calcular las probabilidades para cada observacion

# $P_i = P(x_i) = \frac{1}{1 + e^{-\beta\cdot x_i}}$

In [13]:
def logitprobs(X, beta):
    import numpy as np
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi = list(range(1, n_rows+1))
    expon = list(range(1, n_rows+1))
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex = X[i][j] * beta[j]
            expon[i] = ex + expon[i]
        with np.errstate(divide='ignore', invalid='ignore'):
            pi[i] = 1/(1+np.exp(-expon[i]))
    return pi

## Calcular la matriz diagonal W

## $W = diag(P_i \cdot (1 - P_i))_{i=1}^n$

In [3]:
def findW(pi):
    import numpy as np
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n)
    for i in range(n):
        print(i)
        W[i, i] = pi[i] * (1-pi[i])
        W[i, i].astype(float)
    return W

## Obtener la solucion de la funcion logistica. Usando la funcion de Newton-Raphson

# $ \beta_{n+1} = \beta_n -\frac{f(\beta_n)}{f'(\beta_n)}$
# $ f(\beta) = X(Y - P)$
# $ f'(\beta) = XWX^T$

In [4]:
def logistics(X, Y, limit):
    import numpy as np
    from numpy import linalg
    n_rows = np.shape(X)[0]
    bias = np.ones(n_rows).reshape(n_rows, 1)
    X_new = np.append(X, bias, axis=1)
    n_cols = np.shape(X_new)[1]
    beta = np.zeros(n_cols).reshape(n_cols, 1)
    root_dif = np.array(range(1,n_cols+1)).reshape(n_cols, 1)
    iter_i = 10000
    while (iter_i > limit):
        print(f'{iter_i}, {limit}')
        pi = logitprobs(X_new, beta)
        print(pi)
        W = findW(pi)
        print(W)
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose())
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new))
        root_dif = np.array(np.linalg.inv(den)*num)
        beta = beta + root_dif
        print(beta)
        iter_i = np.sum(root_dif*root_dif)
        print(iter_i)
        ll = linkelihood(Y, pi)
    return beta 



## Comprobacion experimental

In [5]:
import numpy as np

In [6]:
X = np.array(range(10)).reshape(10, 1)
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [7]:
Y = [0,0,0,0,1,0,1,0,1,1]

In [8]:
bias = np.ones(10).reshape(10,1)
X_new = np.append(X, bias, axis=1)

In [9]:
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [18]:
a = logistics(X, Y, 0.00001)

10000, 1e-05
[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
[[ 0.43636364]
 [-2.36363636]]
5.777190082644626
5.777190082644626, 1e-05
[array([0.08598797]), array([0.12705276]), array([0.18378532]), array([0.2583532]), array([0.35019508]), array([0.45467026]), array([0.56329497]), array([0.66616913]), array([0.75533524]), array([0.82687453])]
0
1
2
3
4
5

# $Y = 0.66220827*X -3.69557172$

## Con el paquete statsmodel de Python

In [19]:
import statsmodels.api  as sm

In [20]:
logit_model = sm.Logit(Y, X_new)

In [21]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [23]:
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Logit
===============================================================
Model:              Logit            Pseudo R-squared: 0.360   
Dependent Variable: y                AIC:              12.6202 
Date:               2021-09-17 11:44 BIC:              13.2254 
No. Observations:   10               Log-Likelihood:   -4.3101 
Df Model:           1                LL-Null:          -6.7301 
Df Residuals:       8                LLR p-value:      0.027807
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     6.0000                                     
-----------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
-----------------------------------------------------------------
x1        0.6622     0.4001    1.6551   0.0979   -0.1220   1.4464
const    -3.6956     2.2889   -1.6145   0.1064   -8.1818   0.7906
===============================================================

"""